Cell 1 – Title & Overview (Markdown)
# Bass MIDI Generator (v0.1 → v0.2)

This notebook generates **EDM/house-style bassline MIDI patterns** using Python and exports them as `.mid` files you can drag directly into your DAW (Ableton, FL Studio, Logic, etc.).

The goals of this project are to:

- Quickly create **usable bassline ideas** for house/EDM tracks.
- Let you control **key, BPM, number of bars, and basic pattern behavior**.
- Build a foundation that can be extended with **groove, syncopation, and presets** in later versions.

> **Tip:** The workflow is:
> 1. Run all cells from top to bottom once.
> 2. Enter your desired settings in the main generator cell near the end.
> 3. Download the `.mid` file and drag it into your DAW.

Cell 2 – Imports & Global Setup


In [ ]:
# Bass MIDI Generator - Setup & Imports (Scaffold)
# ------------------------------------------------
# This cell:
# - Installs mido if missing
# - Imports required libraries (MIDI handling, random, typing, paths).
# - Defines global constants used throughout the notebook.
# - Sets up a default output directory for generated MIDI files.


# --------------------
# Install mido
# --------------------



try:
    import mido
except ModuleNotFoundError:
    print('Installing mido...')
    !pip install mido
    import mido


# ------------------------------------------
# Imports
# ------------------------------------------


from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Optional

import random


#---------------------
# Global MIDI defaults
#---------------------


# Default tempo (beats per minute) for the project.
DEFAULT_BPM: int = 125

# Ticks per beat for MIDI timing.
# 480 is a common PPQ (pulses per quarter note) value.
TICKS_PER_BEAT: int = 480

# Default MIDI velocity for bass notes (0-127).
DEFAULT_VELOCITY: int = 96

# Default MIDI channel for bass (0-15). 0 is fine for most use cases.
DEFAULT_CHANNEL: int = 0


# --------------------
# Reproducibility
# --------------------


# Seed for random-based generators (can change if you want more variety).
RANDOM_SEED: int = 42
random.seed(RANDOM_SEED)


# --------------------
# Projects paths
# --------------------


# Root directory for this notebook/project.
PROJECT_ROOT = Path('.').resolve()

# Directory where generated bass MIDI files will be saved.
OUTPUT_DIR = PROJECT_ROOT / 'bass_midi_output'
OUTPUT_DIR.mkdir(exist_ok=True)

print('Bass MIDI Generator setup complete.')
print(f'Project root: {PROJECT_ROOT}')
print(f'Output directory: {OUTPUT_DIR}')
print(f'Defaults -> BPM: {DEFAULT_BPM}, TICKS_PER_BEAT: {TICKS_PER_BEAT}, '
      f'VELOCITY: {DEFAULT_VELOCITY}, CHANNEL: {DEFAULT_CHANNEL}')



Bass MIDI Generator setup complete.
Project root: /content
Output directory: /content/bass_midi_output
Defaults -> BPM: 125, TICKS_PER_BEAT: 480, VELOCITY: 96, CHANNEL: 0


Cell 3 – Core User Configuration (v0.1)

In [ ]:
# Core User Configuration (v0.1)
# ------------------------------
# Basic settings for the bassline generator.
# These will be used by the v0.1 generator.
# you can adjust any of these before running cell 8.


# Musical key for the bassline (e.g., 'A', 'G', 'F')
key: str = 'A'


# Scale mode - v0.1 will support 'minor' only for now
scale_mode: str = 'minor'


# Tempo in BPM
bpm: int = 125


# Length of pattern in bars
num_bars: int = 4


# Starting bass note in MIDI (A2 = 45 is a good default)
root_midi_note: int = 45


# Filename for the v0.1 bassline output
output_filename_v01: str = 'bass_pattern_v0_1.mid'

print('v0.1 user configuration loaded.')
print(f'Key: {key}, Scale: {scale_mode}, BPM: {bpm}, Bars: {num_bars}, Root: {root_midi_note}')





v0.1 user configuration loaded.
Key: A, Scale: minor, BPM: 125, Bars: 4, Root: 45


Cell 4 – Scale & Note Utilities

In [ ]:
# Scale & Note Utilities
# ----------------------
# Helper functions to handle key/scale and note choices for the bassline.
#
# v0.1 goal:
# - Support basic minor scales for common keys.
# - Provide a way to get MIDI notes for the chosen key/scale.
# - Provide helpers to choose a good root note for the bass range.
# - Simple 5th / octaves helpers for future use.


from typing import List


# -----------------------
# Key -> MIDI mapping
# -----------------------
# We use a simple mapping of note names to MIDI numbers in a 'reference octave'.
# Here we choose C3-B3 as the base range (MIDI 48-59).
#
# You can always shift up/down by octaves later:
#  pitch + 12 -> one octave up
#  pitch - 12 -> one octave down


KEY_TO_MIDI_BASE = {
     'C': 48,  # C3
     'D': 50,  # D3
     'E': 52,  # E3
     'F': 53,  # F3
     'G': 55,  # G3
     'A': 57,  # A3
     'B': 59,  # B3
}


# Natural minor scale intervals (relative to root) in semitones.
# For example, A minor: A, B, C, D, E, F, G, A
NATURAL_MINOR_INTERVALS = [0, 2, 3, 5, 7, 8, 10, 12]


def get_scale_midi_notes(key: str, scale_mode: str, octave_offset: int = -1) -> List[int]:
    '''
    Given a key name (e.g., 'A') and a scale mode (currently 'minor'),
    return a list of MIDI note numbers representing the scale.

    octave_offset shifts the base octave:
      0 -> use the base octave (C3-B3)
     -1 -> one octave lower (C2-B2)  [good for bass]
      1 -> one octave higher, etc.
    '''
    key_upper = key.upper()

    if key_upper not in KEY_TO_MIDI_BASE:
        raise ValueError(f'Unsupported key: {key}. Supported keys: {list(KEY_TO_MIDI_BASE.keys())}')

    if scale_mode.lower() != 'minor':
        # for v0.1 we only support natural minor
        raise ValueError(f"Unsupported scale mode: {scale_mode}. v0.1 supports 'minor' only.")

    base_root = KEY_TO_MIDI_BASE[key_upper]
    root_with_offset = base_root + (12 * octave_offset)

    scale_notes = [root_with_offset + interval for interval in NATURAL_MINOR_INTERVALS]
    return scale_notes


def choose_root_note(scale_notes: List[int], root_midi_note: int) -> int:
    '''
    Given a list of scale notes and a desired root_midi_note,
    pick the scale note that is closest to root_midi_note.

    This lets you say:
      - 'I want my bass around A2 (45)' and then
      - snap that to the nearest note in the chosen scale.
    '''
    if not scale_notes:
        raise ValueError('scale_notes is empty.')

    closest = min(scale_notes, key=lambda n: abs(n - root_midi_note))
    return closest


def get_fifth(pitch: int) -> int:
    '''Return the perfect fifth above the given pitch (pitch + 7 semitones).'''
    return pitch + 7


def get_octave_up(pitch: int) -> int:
    '''Return the pitch one octave above (pitch + 12).'''
    return pitch + 12


def get_octave_down(pitch: int) -> int:
    '''Return the pitch one octave below (pitch - 12).'''
    return pitch - 12



print('Scale & note utilities ready.')



Scale & note utilities ready.


Cell 5 – Basic Timing Grid (v0.1)

In [ ]:
# Timing Grid for v0.1 Bass Patterns
# ----------------------------------
# Defines when notes occur in time for simple bassline patterns.
#
# v0.1 design:
# - Assume 4/4 time.
# - Place one note on each beat.
# - Keep things predictable and easy to reason about.
#
# The main helper:
#   build_basic_timing_grid(num_bars: int) -> List[tuple[int, float]]
#
# Each element is:
#   ( bar_index, beat_position_in_bar)
#
# Example for num_bars = 2:
#    [
#      (0, 0.0), (0, 1.0), (0, 2.0), (0, 3.0),
#      (1, 0.0), (1, 1.0), (1, 2.0), (1, 3.0)
#    ]
#
# A small helper is also provided to convert (bar, beat_in_bar)
# into an absolute beat index from the start of the pattern,
# which will be handy when turning this into MIDI ticks later.



from typing import List, Tuple



# For v0.1 we assume standard house/EDM 4/4 time.
BEATS_PER_BAR: int = 4


def build_basic_timing_grid(num_bars: int) -> List[Tuple[int, float]]:

    '''
    Build a simple timing grid for v0.1 bass patterns.

    Args:
        num_bars: Number of bars in the pattern (must be >= 1).
    Returns:
        A list of (bar_index, beat_position_in_bar) tuples.
        - bar_index starts at 0 up to num_bars - 1.
        - beat_position_in_bar is 0.0, 1.0, 2.0, 3.0 for 4/4.
    '''
    if num_bars <= 0:
        raise ValueError(f'num_bars must be positive, got {num_bars}.')

    grid: List[Tuple[int, float]] = []

    for bar_idx in range(num_bars):
        for beat in range(BEATS_PER_BAR):
            grid.append((bar_idx, float(beat)))

    return grid



def bar_beat_to_absolute_beats(
    bar_index: int,
    beat_in_bar: float,
    beats_per_bar: int = BEATS_PER_BAR
) -> float:
    '''
    Convert (bar_index, beat_in_bar) into an absolute beat position
    from the start of the pattern.

    Example:
        bar_index=0, beat_in_bar=0.0 -> 0.0
        bar_index=0, beat_in_bar=1.0 -> 1.0
        bar_index=1, beat_in_bar=0.0 -> 4.0  (assuming 4 beats per bar)

    This will be used later when turning the timing grid into MIDI
    ticks based on BPM and TICKS_PER_BEAT.
    '''
    if bar_index < 0:
        raise ValueError(f'bar_index must be >= 0, got {bar_index}.')
    if beat_in_bar < 0:
        raise ValueError(f'beat_in_bar must be >= 0, got {beat_in_bar}.')

    return bar_index * beats_per_bar + beat_in_bar



print('Basic timing grid helpers ready (v0.1).')












Basic timing grid helpers ready (v0.1).


Cell 6 – v0.1 Bassline Generator

In [ ]:
# v0.1 Bassline Generator - Core Logic
# ----------------------------------------------
# First version: simple, root-focused basslines using the basic timing grid.
#
# Design for v0.1:
# - 4/4 time
# - One note on every beat
# - All notes use a single bass root (aligned to the chosen key/scale)
# - Timing expressed in both beats and ticks so the MIDI writer (Cell 7)
#   can work directly in ticks.


from typing import List, Dict


def generate_bassline_v01(
    key: str,
    scale_mode: str,
    num_bars: int,
    bpm: int,
    root_midi_note: int,
    note_length_beats: float = 1.0,
    velocity: int = DEFAULT_VELOCITY,
    channel: int = DEFAULT_CHANNEL
) -> List[Dict]:
    '''
    Generate a simple v0.1 bassline pattern.

    Args:
        key: Musical key, e.g. 'A', 'G', 'F'.
        scale_mode: Scale mode, v0.1 expects 'minor'.
        num_bars: Number of bars in pattern.
        bpm: Tempo in beats per minute (kept for future use;
             timing grid itself is in beats).
        root_midi_note: Target bass root (e.g., 45 for A2).
        note_length_beats: Duration of each note in beats.
        velocity: MIDI velocity for each note.
        channel: MIDI channel number.

    Returns:
        A list of note event dictionaries. Each event includes:
        - 'start_beat'
        - 'duration_beats'
        - 'start_tick'
        - 'duration_ticks'
        - 'pitch'
        - 'velocity'
        - 'channel'
    '''


    # 1) Build the timing grid: one note on every beat across all bars.
    timing_grid = build_basic_timing_grid(num_bars)

    # 2) Get the scale notes and choose a good bass root in that scale.
    scale_notes = get_scale_midi_notes(
        key=key,
        scale_mode=scale_mode,
        octave_offset=-1
    )
    bass_root = choose_root_note(scale_notes, root_midi_note)

    note_events: List[Dict] = []

    # 3) For each (bar, beat_in_bar), create a note event.
    for bar_idx, beat_in_bar in timing_grid:
        start_beat = bar_beat_to_absolute_beats(
            bar_index=bar_idx,
            beat_in_bar=beat_in_bar
        )
        duration_beats = note_length_beats

        # 4) Convert beats into ticks using the global TICKS_PER_BEAT.
        start_tick = int(start_beat * TICKS_PER_BEAT)
        duration_ticks = int(duration_beats * TICKS_PER_BEAT)

        # 5) Build the note event dictionary.
        note_events.append(
            {
                'start_beat': start_beat,
                'duration_beats': duration_beats,
                'start_tick': start_tick,
                'duration_ticks': duration_ticks,
                'pitch': bass_root,
                'velocity': velocity,
                'channel': channel,

            }
        )


    return note_events



print('v0.1 bassline generator ready.')

v0.1 bassline generator ready.


Cell 7 – MIDI Writer Utility

In [ ]:
# MIDI Writer Utility
# -------------------
# Converts a list of note events into a `.mid` file.
#
# Expects note_events in the format produced by generate_bassline_v01/v02:
#    {
#        'start_beat': float,
#        'duration_beats': float,
#        'start_tick': int,
#        'duration_ticks': int,
#        'pitch': int,
#        'velocity': int,
#        'channel': int
#    }
#
# This function:
# - Creates a MIDI file with the global TICKS_PER_BEAT.
# - Sets the tempo based on BPM.
# - Writes note_on/note_off events with correct delta times.
# - Saves the file into OUTPUT_DIR defined in Cell 2.

from typing import List, Dict
import mido


def write_bassline_to_midi(
    note_events: List[Dict],
    output_filename: str,
    bpm: int,
    ticks_per_beat: int = TICKS_PER_BEAT
) -> None:
    """
    Write a list of bassline note events to a MIDI file.

    Args:
        note_events: List of note dictionaries (see format above).
        output_filename: Name of the MIDI file to create (e.g., 'bass_v01.mid')
        bpm: Tempo in beats per minute.
        ticks_per_beat: PPQ resolution for the MIDI file (default from globals).

    Returns:
        None. Writes a .mid file to OUTPUT_DIR.
    """

    if not note_events:
        raise ValueError('note_events is empty; nothing to write.')

    # 1) Create a new MIDI file and track.
    mid = mido.MidiFile(ticks_per_beat=ticks_per_beat)
    track = mido.MidiTrack()
    mid.tracks.append(track)

    # 2) Set the tempo meta message based on BPM.
    tempo = mido.bpm2tempo(bpm)
    track.append(mido.MetaMessage('set_tempo', tempo=tempo, time=0))

    # 3) Build a list of (absolute_tick, message) pairs for note_on and note_off.
    #    This avoids drift when patterns are dense or notes overlap.
    timed_messages = []

    for ev in note_events:
        start_tick = int(ev['start_tick'])
        duration_ticks = int(ev['duration_ticks'])
        pitch = int(ev['pitch'])
        velocity = int(ev['velocity'])
        channel = int(ev['channel'])

        # note_on at start_tick
        note_on_msg = mido.Message(
            'note_on',
            note=pitch,
            velocity=velocity,
            time=0,           # fill later with delta
            channel=channel,
        )
        timed_messages.append((start_tick, note_on_msg))

        # note_off at start_tick + duration_ticks
        note_off_msg = mido.Message(
            'note_off',
            note=pitch,
            velocity=0,
            time=0,           # fill later with delta
            channel=channel,
        )
        timed_messages.append((start_tick + duration_ticks, note_off_msg))

    # 4) Sort all messages by their absolute tick time.
    timed_messages.sort(key=lambda pair: pair[0])

    # 5) Convert absolute ticks to delta times and add to the track.
    current_tick = 0
    for abs_tick, msg in timed_messages:
        delta = abs_tick - current_tick
        if delta < 0:
            # Should not happen, but guard against bad data.
            delta = 0
        msg.time = delta
        track.append(msg)
        current_tick = abs_tick

    # 6) Resolve full output path and save.
    output_path = OUTPUT_DIR / output_filename
    mid.save(output_path)

    # Optional: small debug print for pattern length in beats/bars.
    # This helps verify that num_bars is behaving as expected.
    last_beat = max(ev['start_beat'] + ev['duration_beats'] for ev in note_events)
    approx_bars = last_beat / 4.0  # assuming 4/4
    print(f'Pattern length (approx): {last_beat:.2f} beats ≈ {approx_bars:.2f} bars')
    print(f'MIDI file written: {output_path}')





Cell 8 – v0.1 Run Cell Bassline Generator

In [ ]:
# Run v0.1 Bassline Generator
# ---------------------------
# This cell will:
# - Use the core configuration from Cell 3
# - Generate the v0.1 bassline note events
# - Write them to a MIDI file using the writer utility
# - Print a simple summary for the user


# 1) Generate the bassline note events.
note_events_v01 = generate_bassline_v01(
    key=key,
    scale_mode=scale_mode,
    num_bars=num_bars,
    bpm=bpm,
    root_midi_note=root_midi_note,
)


# 2) Write them to a MIDI file.
write_bassline_to_midi(
    note_events=note_events_v01,
    output_filename=output_filename_v01,
    bpm=bpm,
)


# 3) Print a quick summary.
print(f'Generated v0.1 bassline: {output_filename_v01}')
print(f'Key: {key} {scale_mode}, BPM: {bpm}, Bars: {num_bars}')
print(f'Number of notes: {len(note_events_v01)}')


Pattern length (approx): 16.00 beats ≈ 4.00 bars
MIDI file written: /content/bass_midi_output/bass_pattern_v0_1.mid
Generated v0.1 bassline: bass_pattern_v0_1.mid
Key: A minor, BPM: 125, Bars: 4
Number of notes: 16


Cell 9 – Extended Config for v0.2 (Pattern & Groove)

In [ ]:
# Extended Configuration for v0.2
# -------------------------------
# These settings control pattern style, density, octave behavior,
# variation, and groove for the advanced bassline generator.


# Pattern / rhythmic behavior (v0.2 uses this heavily)
pattern_style: str = 'four_on_floor'


# How busy the bassline is (0.0 = sparse, 1.0 = very dense)
note_density: float = 0.75


# Whether to stay in one octave or occasionally jump
octave_mode: str = 'single_octave'


# Amount of musical variation per bar
variation_amount: float = 0.25


# Timing/velocity looseness for groove feel
groove_amount: float = 0.15


# Output file name for v0.2
output_filename_v02: str = 'bass_pattern_v0_2.mid'


# Bundle config for convenience
bass_config_v02 = {
    'pattern_style': pattern_style,
    'note_density': note_density,
    'octave_mode': octave_mode,
    'variation_amount': variation_amount,
    'groove_amount': groove_amount,
    'output_filename_v02': output_filename_v02
}


print('v0.2 extended configuration loaded.')
print(f'Pattern: {pattern_style}, Density: {note_density}, Octave: {octave_mode}')
print(f'Variation: {variation_amount}, Groove: {groove_amount}')
print(f'Output file: {output_filename_v02}')



v0.2 extended configuration loaded.
Pattern: four_on_floor, Density: 0.75, Octave: single_octave
Variation: 0.25, Groove: 0.15
Output file: bass_pattern_v0_2.mid


Cell 10 – Pattern Style Helpers (v0.2)

In [ ]:
# Pattern Style Helpers (v0.2)
# ----------------------------
# Builds timing grids for differnt bassline styles.
#
# Returns a list of (bar_index, beat_position_in_bar) tuples,
# similar to build_basic_timing_grid, but shaped by style and note_density.


from typing import List, Tuple


def _per_bar_positions_for_style(style: str) -> List[float]:
    '''
    Internal helper: return beat positions (within a single bar)
    for a given style, before note_density is applied.
    '''
    style_lower = style.lower()


    if style_lower == 'four_on_floor':
        # Strong notes on each beat: 1, 2, 3, 4
        return [0.0, 1.0, 2.0, 3.0]


    if style_lower == 'offbeat':
        # Notes on the 'ands': 1&, 2&, 3&, 4&
        return [0.5, 1.5, 2.5, 3.5]


    if style_lower == 'rolling':
        # Simple 1/8-note rolling pattern across the bar
        return [i * 0.5 for i in range(8)]


    if style_lower == 'minimal':
        # Sparse: only a couple of strong beats
        return [0.0, 2.0]


    # Fallback: treat unknown styles as four_on_floor
    return [0.0, 1.0, 2.0, 3.0]


def build_timing_grid_for_style(
    style: str,
    num_bars: int,
    note_density: float,
) -> List[Tuple[int, float]]:
    '''
    Build a timing grid for the given style and density.

    Args:
        style: Pattern style name (e.g., 'four_on_floor', 'offbeat', 'rolling', 'minimal'),
        num_bars: Number of bars in the pattern.
        note_density: 0.0 to 1.0, controls how many of the potential
                      positions are actually used.

    Returns:
        List of (bar_index, beat_position_in_bar) tuples.
    '''


    if num_bars <= 0:
        raise ValueError(f'num_bars must be positive, got {num_bars}.')


    # Clamp density to [0.0, 1.0] for safety.
    density = max(0.0, min(1.0, note_density))


    base_positions = _per_bar_positions_for_style(style)
    grid: List[Tuple[int, float]] = []


    for bar_idx in range(num_bars):
        for beat_pos in base_positions:
            # Randomly keep or drop this note based on density.
            if random.random() <= density:
                grid.append((bar_idx, beat_pos))


    # Edge case: if density was very low and nothing survived,
    # ensure at least one note per bar by forcing the first position.
    if not grid:
        for bar_idx in range(num_bars):
            if base_positions:
                grid.append((bar_idx, base_positions[0]))


    return grid


print('Pattern style helpers ready for v0.2')



Pattern style helpers ready for v0.2


Cell 11 – Groove & Humanization Helpers

In [ ]:
# Groove & Humanization Helpers
# -----------------------------
# Adds timing and velocity 'feel' to note events for v0.2.
#
# Expects note_events in the same general format as v0.1/v0.2:
#   {
#        'start_beat': float,
#        'duration_beats': float,
#        'start_tick': int,
#        'duration_ticks': int,
#        'pitch': int,
#        'velocity': int,
#        'channel': int,
#   }
#
# These helpers operate in BEATS first, then recalc TICKS
# using the global TICKS_PER_BEAT from cell 2.


from typing import List, Dict


def apply_timing_groove(
    note_events: List[Dict],
    groove_amount: float,
    beats_per_bar: int = BEATS_PER_BAR,
    max_swing_offset_beats: float = 0.04,
    random_micro_offset_beats: float = 0.02
) -> List[Dict]:
    '''
    Apply timing groove to note_events.

    groove_amount: 0.0 to 1.0
      - 0.0 -> no timing change (perfect grid)
      - 0.3 -> subtle
      - 0.6 -> noticeable
      - 1.0 -> strong, experimental
    '''


    if groove_amount <= 0.0:
        return note_events


    # Clamp to [0.0, 1.0] for safety.
    g = max(0.0, min(1.0, groove_amount))


    for ev in note_events:
        start_beat = ev.get('start_beat', 0.0)


        # Derive bar index and beat position within the bar.
        bar_index = int(start_beat // beats_per_bar)
        beat_in_bar = start_beat - (bar_index * beats_per_bar)


        # Determine if this note is on an 8th-note grid (0.0, 0.5, 1.0, 1.5, ...)
        # We'll swing the 'off' 8ths (0.5, 1.5, 2.5, 3.5, etc.).
        is_eighth = abs((beat_in_bar * 2.0) - round(beat_in_bar * 2.0)) < 1e-6
        is_off_eighth = is_eighth and int(round(beat_in_bar * 2.0)) % 2 == 1


        timing_offset = 0.0


        if is_off_eighth:
            # Swing the off-beats: push them later a bit.
            timing_offset += max_swing_offset_beats * g


        # Add a small random micro-timing jitter (both ahead and behind).
        if random_micro_offset_beats > 0.0:
            jitter_range = random_micro_offset_beats * g
            timing_offset += random.uniform(-jitter_range, jitter_range)


        # Apply offset and clamp so we don't go negative in time.
        new_start_beat = max(0.0, start_beat + timing_offset)


        ev['start_beat'] = new_start_beat
        ev['start_tick'] = int(new_start_beat * TICKS_PER_BEAT)


    return note_events


def apply_velocity_variation(
    note_events: List[Dict],
    variation_amount: float,
    base_velocity_key: str = 'velocity',
    max_variation_range: int = 18,
) -> List[Dict]:
    '''
    Apply velocity variation to note_events.

    variation_amount: 0.0 to 1.0
      - 0.0 -> no change
      - 0.3 -> subtle
      - 0.6 -> noticeable accents
      - 1.0 -> very dynamic
    '''


    if variation_amount <= 0.0:
        return note_events


    v = max(0.0, min(1.0, variation_amount))


    for ev in note_events:
        base_vel = int(ev.get(base_velocity_key, DEFAULT_VELOCITY))


        # Random variation around the base velocity.
        # We also add a tiny beat-based accent for beat 1 of each bar.
        start_beat = ev.get('start_beat', 0.0)
        bar_index = int(start_beat // BEATS_PER_BAR)
        beat_in_bar = start_beat - (bar_index * BEATS_PER_BAR)


        # Stronger accents on the '1' of the bar.
        if abs(beat_in_bar - 0.0) < 1e-6:
            accent = int(4 * v)
        else:
            accent = 0


        # Random +/- variation scaled by v.
        random_range = int(max_variation_range * v)
        random_delta = random.randint(-random_range, random_range)


        new_vel = base_vel + accent + random_delta
        new_vel = max(1, min(127, new_vel))


        ev[base_velocity_key] = new_vel


    return note_events


print('Groove & humanization helpers ready for v0.2')



Groove & humanization helpers ready for v0.2


Cell 12 – v0.2 Bassline Generator (Advanced Logic)

In [ ]:
# v0.2 Bassline Generator - Advanced Logic
# ---------------------------------------------------
# Uses:
# - Scale utilities (Cell 4)
# - Pattern style helpers (Cell 10)
# - Groove & humanization (Cell)
#
# Returns a list of note event dicts, similar to v0.1 but with:
# - Style-based timing
# - Optional octave jumps
# - Variation in note choice
# - Groove and velocity movement


from typing import List, Dict


def generate_bassline_v02(
    key: str,
    scale_mode: str,
    num_bars: int,
    bpm: int,
    root_midi_note: int,
    pattern_style: str,
    note_density: float,
    octave_mode: str,
    variation_amount: float,
    groove_amount: float,
    note_length_beats: float = 1.0,
    base_velocity: int = DEFAULT_VELOCITY,
    channel: int = DEFAULT_CHANNEL,
) -> List[Dict]:
    '''
    Generate a v0.2 bassline pattern with style, variation, and groove.

    Args:
        key: Musical key, e.g. 'A', 'G', 'F'.
        scale_mode: e.g. 'minor'.
        num_bars: Number of bars in the pattern.
        bpm: Tempo in BPM (mainly used downstream for MIDI tempo).
        root_midi_note: Target bass register (e.g., 45 for A2).
        pattern_style: Pattern style name (four_on_floor, offbeat, rolling, minimal).
        note_density: 0.0 to 1.0, controls how many candidate hits are used.
        octave_mode: 'single_octave' or 'two_octaves'.
        variation_amount: 0.0 to 1.0, controls pitch/octave variation and dynamics.
        groove_amount: 0.0 to 1.0, controls timing groove intensity.
        note_length_beats: Base note length in beats.
        base_velocity: Base MIDI velocity.
        channel: MIDI channel.


    Returns:
        List of note event dicts with:
        - start_beat, duration_beats
        - start_tick, duration_ticks
        - pitch, velocity, channel
    '''


    # 1) Build the style-based timing grid.
    timing_grid = build_timing_grid_for_style(
        style=pattern_style,
        num_bars=num_bars,
        note_density=note_density,
    )


    if not timing_grid:
        # Safety fallback: if style/density somehow produced nothing,
        # fall back to the basic v0.1 grid.
        timing_grid = build_basic_timing_grid(num_bars)


    # 2) Get scale notes and pick a good bass root.
    scale_notes = get_scale_midi_notes(
        key=key,
        scale_mode=scale_mode,
        octave_offset=-1
    )
    bass_root = choose_root_note(scale_notes, root_midi_note)

    note_events: List[Dict] = []


    # Clamp variation for safety.
    v_amt = max(0.0, min(1.0, variation_amount))


    for bar_idx, beat_in_bar in timing_grid:
        # Base pitch is the bass root.
        pitch = bass_root


        # Occasional variation in pitch based on variation_amount.
        # Keep this relatively subtle for v0.2
        if v_amt > 0.0:
            # Small chance to use another scale note near the root.
            if random.random() < v_amt * 0.4:
                # Choose a scale note near bass_root.
                close_notes = sorted(
                    scale_notes,
                    key=lambda n: abs(n - bass_root)
                )
                # Take one of the three closest notes if possible.
                candidates = close_notes[:3] if len(close_notes) >= 3 else close_notes
                if candidates:
                    pitch = random.choice(candidates)


            # Optional octave behavior.
            if octave_mode == 'two_octaves' and random.random() < v_amt * 0.3:
                # randomly go up or down an octave.
                if random.random() < 0.5:
                    pitch = get_octave_up(pitch)
                else:
                    pitch = get_octave_down(pitch)


        # Compute absolute start beat and duration.
        start_beat = bar_beat_to_absolute_beats(
            bar_index=bar_idx,
            beat_in_bar=beat_in_bar,
        )
        duration_beats = note_length_beats


        start_tick = int(start_beat * TICKS_PER_BEAT)
        duration_ticks = int(duration_beats * TICKS_PER_BEAT)


        note_events.append(
            {
                'start_beat': start_beat,
                'duration_beats': duration_beats,
                'start_tick': start_tick,
                'duration_ticks': duration_ticks,
                'pitch': pitch,
                'velocity': base_velocity,
                'channel': channel,
            }
        )


    # 3) Apply timing groove.
    note_events = apply_timing_groove(
        note_events=note_events,
        groove_amount=groove_amount,
        beats_per_bar=BEATS_PER_BAR,
    )


    # 4) Apply velocity variation (reuse variation_amount as intensity).
    note_events = apply_velocity_variation(
        note_events=note_events,
        variation_amount=v_amt,
        base_velocity_key='velocity',
    )


        # 3) Apply timing groove.
    note_events = apply_timing_groove(
        note_events=note_events,
        groove_amount=groove_amount,
        beats_per_bar=BEATS_PER_BAR,
    )

    # 4) Apply velocity variation (reuse variation_amount as intensity).
    note_events = apply_velocity_variation(
        note_events=note_events,
        variation_amount=v_amt,
        base_velocity_key='velocity',
    )

    # 5) Clamp any notes that run past the end of the pattern.
    #    This keeps the total length to num_bars exactly.
    pattern_end_beat = num_bars * BEATS_PER_BAR
    for ev in note_events:
        start = ev.get('start_beat', 0.0)
        dur = ev.get('duration_beats', 0.0)
        end = start + dur
        if end > pattern_end_beat:
            new_dur = max(0.0, pattern_end_beat - start)
            ev['duration_beats'] = new_dur
            ev['duration_ticks'] = int(new_dur * TICKS_PER_BEAT)

    return note_events



print('v0.2 bassline generator ready.')








v0.2 bassline generator ready.


Cell 13 – Presets & Quick A/B Comparison


In [ ]:
# Presets & v0.1 vs v0.2 Comparison
# ---------------------------------
# Defines named presets and supports quick comparison between generators.
#
# This cell provides:
# - bass_presets: a dictionary of named bassline presets.
# - generate_v01_and_v02_for_preset: helper for quick A/B in your DAW.
#
# Usage example (after running all previous cells):
#   generate_v01_and_v02_for_preset('classic_house')
#
# This will:
#   - Generate a v0.1 bassline MIDI file for that preset.
#   - Generate a v0.2 bassline MIDI file for that preset.
#   - Print a summary with filename and note counts.


from typing import Dict, List


# -----------------------------------
# Preset definitions
# -----------------------------------
# Each preset includes:
# - Core musical settings (used by both v0.1 and v0.2)
# - v0.2 pattern and groove settings
# - Output file names for v0.1 and v0.2
# - A short human-readable description


bass_presets: Dict[str, Dict[str, object]] = {
    'classic_house': {
        'description': 'Solid four-on-the-floor bass in a minor with subtle movement and groove.',
        'key': 'A',
        'scale_mode': 'minor',
        'bpm': 125,
        'num_bars': 4,
        'root_midi_note': 45,   # A2
        'pattern_style': 'four_on_floor',
        'note_density': 0.7,
        'octave_mode': 'single_octave',
        'variation_amount': 0.25,
        'groove_amount': 0.18,
        'output_filename_v01': 'bass_classic_house_v0_1.mid',
        'output_filename_v02': 'bass_classic_house_v0_2.mid',

    },
    'rolling_bass':{
        'description': 'Rolling 1/8-note bass with moremotion and two-octave range.',
        'key': 'G',
        'scale_mode': 'minor',
        'bpm': 128,
        'num_bars': 4,
        'root_midi_note': 43,   # G2
        'pattern_style': 'rolling',
        'note_density': 0.85,
        'octave_mode': 'two_octaves',
        'variation_amount': 0.5,
        'groove_amount':0.3,
        'output_filename_v01': 'bass_rolling_v0_1.mid',
        'output_filename_v02': 'bass_rolling_v0_2.mid',

    },
    'minimal_tech':{
        'description': 'Sparce, minimal tech bass with subtle variation and light groove,',
        'key': "D",
        'scale_mode': 'minor',
        'bpm': 126,
        'num_bars': 8,
        'root_midi_note': 38,   # D2
        'pattern_style': 'minimal',
        'note_density': 0.35,
        'octave_mode': 'single_octave',
        'variation_amount': 0.3,
        'groove_amount': 0.12,
        'output_filename_v01': 'bass_minimal_tech_v0_1.mid',
        'output_filename_v02': 'bass_minimal_tech_v0_2.mid',

    },
}


def list_bass_presets() -> None:
    '''
    Print available bass presets and their short descriptions.
    '''
    if not bass_presets:
      print('No bass presets defined.')
      return


    print('Available bass presets:')
    for name, cfg in bass_presets.items():
        desc = cfg.get('description', '')
        print(f' - {name}: {desc}')


def generate_v01_and_v02_for_preset(preset_name: str) -> Dict[str, object]:
    '''
    Generate both v0.1 and v0.2 bassline MIDI files for a given preset.

    Args:
        preset_name: Name of the preset key in bass_presets.

    Returns:
        A small summary dictionary with:
        - 'preset_name'
        - 'v01_filename'
        - 'v02_filename'
        - 'v01_note_count'
        - 'v02_note_count'
    '''
    if preset_name not in bass_presets:
        raise ValueError(
            f'Preset "{preset_name}" not found. '
            f'Available presets: {list(bass_presets.keys())}'

        )


    cfg = bass_presets[preset_name]


    key = str(cfg['key'])
    scale_mode = str(cfg['scale_mode'])
    bpm = int(cfg['bpm'])
    num_bars = int(cfg['num_bars'])
    root_midi_note = int(cfg['root_midi_note'])


    pattern_style = str(cfg['pattern_style'])
    note_density = float(cfg['note_density'])
    octave_mode = str(cfg['octave_mode'])
    variation_amount = float(cfg['variation_amount'])
    groove_amount = float(cfg['groove_amount'])


    output_filename_v01 = str(cfg['output_filename_v01'])
    output_filename_v02 = str(cfg['output_filename_v02'])


    # -------------------------
    # Generate v0.1 bassline
    # -------------------------
    note_events_v01: List[Dict] = generate_bassline_v01(
        key=key,
        scale_mode=scale_mode,
        num_bars=num_bars,
        bpm=bpm,
        root_midi_note=root_midi_note
    )


    write_bassline_to_midi(
        note_events=note_events_v01,
        output_filename=output_filename_v01,
        bpm=bpm
    )


    # -------------------------
    # Generate v0.2 bassline
    # -------------------------
    note_events_v02: List[Dict] = generate_bassline_v02(
        key=key,
        scale_mode=scale_mode,
        num_bars=num_bars,
        bpm=bpm,
        root_midi_note=root_midi_note,
        pattern_style=pattern_style,
        note_density=note_density,
        octave_mode=octave_mode,
        variation_amount=variation_amount,
        groove_amount=groove_amount,
    )


    write_bassline_to_midi(
        note_events=note_events_v02,
        output_filename=output_filename_v02,
        bpm=bpm
    )


    # -------------------------
    # Summary
    # -------------------------
    v01_count = len(note_events_v01)
    v02_count = len(note_events_v02)


    print('--------------------------------------------------')
    print(f'Preset: {preset_name}')
    print(f'Key: {key} {scale_mode}, BPM: {bpm}, Bars: {num_bars}')
    print()
    print(f'v0.1 file: {output_filename_v01} (notes: {v01_count})')
    print(f'v0.2 file: {output_filename_v02} (notes: {v02_count})')
    print('--------------------------------------------------')
    print('Files are saved in:', OUTPUT_DIR)
    print('Drag both into your DAW for quick A/B comparison.')
    print('--------------------------------------------------')


    return {
        'preset_name': preset_name,
        'v01_filename': output_filename_v01,
        'v02_filename': output_filename_v02,
        'v01_note_count': v01_count,
        'v02_note_count': v02_count,
    }


print('Bass presets and A/B comparison helpers ready.')
list_bass_presets()





Bass presets and A/B comparison helpers ready.
Available bass presets:
 - classic_house: Solid four-on-the-floor bass in a minor with subtle movement and groove.
 - rolling_bass: Rolling 1/8-note bass with moremotion and two-octave range.
 - minimal_tech: Sparce, minimal tech bass with subtle variation and light groove,


Cell 14 — Main v0.2 Manual Generator (Run After Cells 1–13)

In [ ]:
# Cell 14 – One-Click v0.2 Bass MIDI Generator (Prompt + Download)
# ----------------------------------------------------------------
# Run this cell, answer the prompts (key, scale, bars, BPM, style),
# and it will generate & download a v0.2 bassline MIDI file.

from datetime import datetime
import random

# Try to import Colab's files helper (safe if not in Colab)
try:
    from google.colab import files  # type: ignore
    IN_COLAB = True
except Exception:
    IN_COLAB = False


def _prompt_or_default(prompt: str, default: str) -> str:
    """Prompt user for text, with a default value if they just press Enter."""
    text = input(f'{prompt} [{default}]: ').strip()
    return text or default


def _prompt_int_or_default(prompt: str, default: int) -> int:
    """Prompt user for an integer, with a default and simple validation."""
    text = input(f'{prompt} [{default}]: ').strip()
    if not text:
        return default
    try:
        return int(text)
    except ValueError:
        print('Invalid number, using default.')
        return default


print('\n=== One-Click Bass MIDI Generator (v0.2) ===')

# -------------------------
# Core song info (prompts)
# -------------------------

key_v02 = _prompt_or_default('Key', 'A')
scale_mode_v02 = _prompt_or_default('Scale type (major/minor)', 'minor')
num_bars_v02 = _prompt_int_or_default('Number of bars', 8)
bpm_v02 = _prompt_int_or_default('Tempo (BPM)', 126)

# Pattern style choice (with safe default)
pattern_style_v02 = _prompt_or_default(
    'Pattern style (four_on_floor/offbeat/rolling/minimal)',
    'four_on_floor'
)

# -------------------------
# Advanced defaults (no prompt)
# -------------------------
# These stay fixed so beginners don't have to think about them,
# but they still get variation & groove.

root_midi_note_v02 = 45          # A2 - good general bass register
note_density_v02 = 0.75          # how busy the line is
octave_mode_v02 = 'single_octave'
variation_amount_v02 = 0.25
groove_amount_v02 = 0.15

# Don't set a seed here -> new/random-ish pattern each run.
print('\nGenerating bass MIDI clip...')

note_events_v02 = generate_bassline_v02(
    key=key_v02,
    scale_mode=scale_mode_v02,
    num_bars=num_bars_v02,
    bpm=bpm_v02,
    root_midi_note=root_midi_note_v02,
    pattern_style=pattern_style_v02,
    note_density=note_density_v02,
    octave_mode=octave_mode_v02,
    variation_amount=variation_amount_v02,
    groove_amount=groove_amount_v02,
)

# Build a descriptive filename with a timestamp.
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_basename_v02 = 'bass_pattern_v0_2'
output_filename_v02 = (
    f'{output_basename_v02}_'
    f'{key_v02}{scale_mode_v02}_'
    f'{bpm_v02}bpm_'
    f'{num_bars_v02}bars_'
    f'{timestamp}.mid'
)

write_bassline_to_midi(
    note_events=note_events_v02,
    output_filename=output_filename_v02,
    bpm=bpm_v02,
)

midi_path = OUTPUT_DIR / output_filename_v02




# Debug: show actual pattern length in beats and bars
last_beat = max(ev['start_beat'] + ev['duration_beats'] for ev in note_events_v02)
approx_bars = last_beat / 4.0  # assuming 4/4
print(f'Pattern length (approx): {last_beat:.2f} beats ≈ {approx_bars:.2f} bars')



print('----------------------------------------------------')
print('Bass MIDI Generator v0.2 (Prompt Mode)')
print(f'Key: {key_v02} {scale_mode_v02}, BPM: {bpm_v02}, Bars: {num_bars_v02}')
print(f'Pattern style: {pattern_style_v02}')
print(f'Note density: {note_density_v02}, Octave mode: {octave_mode_v02}')
print(f'Variation: {variation_amount_v02}, Groove: {groove_amount_v02}')
print(f'Notes generated: {len(note_events_v02)}')
print(f'File written: {midi_path}')
print('----------------------------------------------------\n')

if IN_COLAB:
    print('Triggering download...')
    files.download(str(midi_path))
else:
    print('Not running in Colab, so no auto-download.')




=== One-Click Bass MIDI Generator (v0.2) ===
Key [A]: A
Scale type (major/minor) [minor]: minor
Number of bars [8]: 8
Tempo (BPM) [126]: 128
Pattern style (four_on_floor/offbeat/rolling/minimal) [four_on_floor]: rolling

Generating bass MIDI clip...
Pattern length (approx): 32.00 beats ≈ 8.00 bars
MIDI file written: /content/bass_midi_output/bass_pattern_v0_2_Aminor_128bpm_8bars_20251211_042629.mid
Pattern length (approx): 32.00 beats ≈ 8.00 bars
----------------------------------------------------
Bass MIDI Generator v0.2 (Prompt Mode)
Key: A minor, BPM: 128, Bars: 8
Pattern style: rolling
Note density: 0.75, Octave mode: single_octave
Variation: 0.25, Groove: 0.15
Notes generated: 53
File written: /content/bass_midi_output/bass_pattern_v0_2_Aminor_128bpm_8bars_20251211_042629.mid
----------------------------------------------------

Triggering download...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Cell 15 – Notebook README & Usage

## Bass MIDI Generator — Notebook Structure & Usage

This notebook generates **EDM/house-style bassline MIDI files** and exports them to the `bass_midi_output` folder for use in any DAW (Ableton, FL Studio, Logic, etc.).

The goals of this project are to:

- Quickly create **usable bassline ideas** for house/EDM tracks.  
- Let you control **key, BPM, number of bars, and basic pattern behavior**.  
- Provide a foundation for future versions with **groove, syncopation, presets, and more advanced generators**.

---

### Versions

- **v0.1**
  - Simple, root-focused basslines.
  - One note per beat using a basic 4/4 timing grid.
  - Good for static, scaffolding-style bass ideas and quick tests.

- **v0.2**
  - Style-based timing grids (four-on-the-floor, offbeat, rolling, minimal).
  - Note density control (how busy the line is).
  - Optional octave movement.
  - Pitch variation, groove, and velocity movement for a more “human” feel.
  - A **one-click, prompt-based generator (Cell 14)** for fast custom basslines.
  - Internal clamping to ensure the pattern **always ends exactly at the requested number of bars**.

---

### Cell Overview (1–14)

- **Cell 1 – Title & Overview**  
  High-level description of the Bass MIDI Generator project, goals, and basic workflow.

- **Cell 2 – Imports & Global Setup**  
  Installs `mido` if needed and imports all libraries.  
  Defines global MIDI defaults (`DEFAULT_BPM`, `TICKS_PER_BEAT`, `DEFAULT_VELOCITY`, `DEFAULT_CHANNEL`), sets the project root, and creates the `bass_midi_output` directory.

- **Cell 3 – Core User Configuration (v0.1)**  
  Basic settings for the original generator:
  - `key`, `scale_mode` (minor), `bpm`, `num_bars`, `root_midi_note`  
  - `output_filename_v01` for the v0.1 MIDI file.  
  Used by the simple v0.1 generator and its run cell.

- **Cell 4 – Scale & Note Utilities**  
  Utilities for:
  - Mapping keys to base MIDI notes (`KEY_TO_MIDI_BASE`).  
  - Building a natural minor scale (`get_scale_midi_notes`).  
  - Choosing a good bass root in the chosen scale (`choose_root_note`).  
  - Simple interval helpers (`get_fifth`, `get_octave_up`, `get_octave_down`).

- **Cell 5 – Basic Timing Grid (v0.1)**  
  Builds a 4/4 timing grid for v0.1:
  - `build_basic_timing_grid(num_bars)` → one note per beat.  
  - `bar_beat_to_absolute_beats(...)` to convert (bar, beat_in_bar) into an absolute beat index.

- **Cell 6 – v0.1 Bassline Generator**  
  `generate_bassline_v01(...)`:
  - Uses the basic grid from Cell 5 and scale utilities from Cell 4.  
  - Creates root-focused bass notes with start times and durations expressed in both beats and ticks.  
  - Returns a list of note event dictionaries with `start_beat`, `duration_beats`, `start_tick`, `duration_ticks`, `pitch`, `velocity`, and `channel`.

- **Cell 7 – MIDI Writer Utility**  
  `write_bassline_to_midi(note_events, output_filename, bpm, ...)`:
  - Takes note events from v0.1 or v0.2.  
  - Creates a MIDI file with the global `TICKS_PER_BEAT`.  
  - Sets tempo from BPM, writes correct `note_on`/`note_off` events with proper delta times.  
  - Saves the file into `bass_midi_output`.  
  - Prints an approximate pattern length in beats and bars for quick verification.

- **Cell 8 – v0.1 Run Cell**  
  Simple end-to-end runner for v0.1:
  - Uses config from Cell 3.  
  - Calls `generate_bassline_v01` and `write_bassline_to_midi`.  
  - Prints a short summary (key, BPM, bars, and note count) plus the output filename.

- **Cell 9 – Extended Configuration for v0.2**  
  Defines higher-level controls used by the v0.2 ecosystem:
  - `pattern_style`, `note_density`, `octave_mode`, `variation_amount`, `groove_amount`.  
  - `output_filename_v02`.  
  These are bundled into `bass_config_v02` and are especially useful for presets (Cell 13) and for future versions that expose more parameters to users.

- **Cell 10 – Pattern Style Helpers (v0.2)**  
  Functions to build style-based timing grids:
  - `_per_bar_positions_for_style(style)` defines per-bar beat positions for:
    - `four_on_floor`, `offbeat`, `rolling`, `minimal` (and a fallback).  
  - `build_timing_grid_for_style(style, num_bars, note_density)`:
    - Applies note density to decide which candidate hits survive.  
    - Ensures at least one note per bar even at very low density.

- **Cell 11 – Groove & Humanization Helpers**  
  Functions to add feel:
  - `apply_timing_groove(note_events, groove_amount, ...)`:
    - Swings off-beat 8th notes and adds micro-timing jitter.  
    - Recomputes `start_tick` using `TICKS_PER_BEAT`.  
  - `apply_velocity_variation(note_events, variation_amount, ...)`:
    - Adds velocity randomness and extra accents on the downbeat of each bar.  

- **Cell 12 – v0.2 Bassline Generator (Advanced Logic)**  
  `generate_bassline_v02(...)`:
  - Uses the style timing grid from Cell 10.  
  - Builds scale notes and chooses a good bass root (Cell 4).  
  - Adds pitch variation and optional octave jumps (single- or two-octave behavior).  
  - Applies timing groove and velocity variation (Cell 11).  
  - **Clamps any notes that would extend past the final bar**, so the pattern length always matches `num_bars` exactly.  
  - Returns a list of note events in the same format expected by the MIDI writer.

- **Cell 13 – Presets & A/B Comparison**  
  - `bass_presets`: dictionary of named presets such as:
    - `classic_house`, `rolling_bass`, `minimal_tech`.  
    Each preset defines key, scale, BPM, bar length, root note, and v0.2 pattern/groove settings, plus output filenames.  
  - `list_bass_presets()` prints available presets and descriptions.  
  - `generate_v01_and_v02_for_preset(preset_name)`:
    - Generates both v0.1 **and** v0.2 files for the chosen preset.  
    - Saves them to `bass_midi_output`.  
    - Prints a summary with filenames and note counts for quick A/B in your DAW.

- **Cell 14 – One-Click v0.2 Bass MIDI Generator (Prompt + Download)**  
  Main **user-facing** generator for v0.2:
  - Asks you a few questions in the console:
    - `Key` (e.g., A, G, F)  
    - `Scale type (major/minor)` (designed for minor right now)  
    - `Number of bars`  
    - `Tempo (BPM)`  
    - `Pattern style` (`four_on_floor`, `offbeat`, `rolling`, `minimal`)  
  - Uses sensible internal defaults for:
    - `root_midi_note_v02`, `note_density_v02`, `octave_mode_v02`, `variation_amount_v02`, `groove_amount_v02`.  
    These are chosen to give musical results without overwhelming the user, but you can edit them in the cell if you want deeper control.  
  - Calls `generate_bassline_v02` to build the note events, then `write_bassline_to_midi` to save them.  
  - Builds a descriptive output filename that includes:
    - Key, scale, BPM, bar count, and a timestamp, e.g.  
      `bass_pattern_v0_2_Aminor_126bpm_8bars_YYYYMMDD_HHMMSS.mid`.  
  - Prints a detailed summary (key, scale, BPM, bars, pattern style, density, octave mode, variation/groove amounts, note count, and file path).  
  - Shows the **actual pattern length in beats and bars** for debugging.  
  - If running in Google Colab, automatically triggers a file download.  
    - If running locally, the file is simply written to `bass_midi_output` and you can drag it into your DAW.

---

### Typical Workflow

1. **First-time setup**  
   - Run the notebook **top-to-bottom once** (Cells 1–13).  
   - This ensures all utilities, generators, and presets are defined.

2. **Fast custom basslines with Cell 14 (recommended)**  
   - Run **Cell 14**.  
   - Answer the prompts for key, scale, number of bars, BPM, and pattern style.  
   - Wait for the generator to print the summary and, in Colab, trigger the download.  
   - Locate the `.mid` file in `bass_midi_output` (or use the downloaded file) and drag it into your DAW.

3. **Preset-based A/B comparisons (Cell 13)**  
   - Call `list_bass_presets()` to see available presets.  
   - Run `generate_v01_and_v02_for_preset("classic_house")` (or another preset name).  
   - This will create both a v0.1 and v0.2 MIDI file so you can compare “simple vs advanced” versions of the same idea in your DAW.

4. **Basic v0.1 patterns (Cell 8)**  
   - If you want a very simple, on-the-grid bassline, adjust the settings in **Cell 3** and run **Cell 8**.  
   - This is useful for scaffolding or layer ideas where you don’t need groove or variation.

---

### Output Location

All generated MIDI files are written to:

```text
bass_midi_output/
